In [53]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
df = pd.read_csv("https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/master/data.csv")
df.head()


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [54]:
df.drop(columns=['id','Unnamed: 32'],inplace=True)

In [77]:
df["diagnosis"].unique()

array(['M', 'B'], dtype=object)

In [55]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

In [56]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [57]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [58]:
import torch

In [60]:
from torch import nn

In [82]:
X_train_tensor = torch.from_numpy(X_train.astype(np.float32))
X_test_tensor = torch.from_numpy(X_test.astype(np.float32))
y_train_tensor = torch.from_numpy(y_train.astype(np.float32))
y_test_tensor = torch.from_numpy(y_test.astype(np.float32))

In [83]:
from torch.utils.data import Dataset,DataLoader

In [84]:
class CustomDataset(Dataset):

    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return self.features.shape[0]   # number of samples

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]



In [85]:
train_dataset=CustomDataset(X_train_tensor,y_train_tensor)
test_dataset=CustomDataset(X_test_tensor,y_test_tensor)

In [93]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [114]:
import torch.nn as nn

class mysimplenn(nn.Module):
  def __init__(self,num_features) -> None:
      super().__init__()
      self.linear=nn.Linear(num_features,128)
      self.relu=nn.ReLU()
      self.linear2=nn.Linear(128,1)
      self.sigmoid=nn.Sigmoid()

  def forward(self,x):
    x=self.linear(x)
    x=self.relu(x)
    x=self.linear2(x)
    x=self.sigmoid(x)
    return x

In [115]:
learning_rate = 0.1
epochs = 250

In [116]:
model=mysimplenn(X_train_tensor.shape[1])

In [117]:
X_train_tensor.shape

torch.Size([455, 30])

In [118]:
optim=torch.optim.SGD(model.parameters(),lr=learning_rate)
loss_function=nn.BCELoss()

In [119]:
for epoch in range(epochs):
  for x,y in train_loader:
    y_pred=model(x)

    loss=loss_function(y_pred,y.view(-1,1))

    optim.zero_grad()
    loss.backward()
    optim.step()

  print(f'epoch: {epoch}, loss: {loss.item()}')

epoch: 0, loss: 0.4011630415916443
epoch: 1, loss: 0.47278666496276855
epoch: 2, loss: 0.03150957077741623
epoch: 3, loss: 0.05399923399090767
epoch: 4, loss: 0.03614341467618942
epoch: 5, loss: 0.06973905861377716
epoch: 6, loss: 0.07954501360654831
epoch: 7, loss: 0.005141128320246935
epoch: 8, loss: 0.01371213048696518
epoch: 9, loss: 0.046037767082452774
epoch: 10, loss: 0.3309773802757263
epoch: 11, loss: 0.014411481097340584
epoch: 12, loss: 0.020135624334216118
epoch: 13, loss: 0.011136299930512905
epoch: 14, loss: 0.03883155435323715
epoch: 15, loss: 0.017095239832997322
epoch: 16, loss: 0.08243941515684128
epoch: 17, loss: 0.35649770498275757
epoch: 18, loss: 0.02535095252096653
epoch: 19, loss: 0.023116962984204292
epoch: 20, loss: 0.05126947909593582
epoch: 21, loss: 0.0159135852009058
epoch: 22, loss: 0.03287065774202347
epoch: 23, loss: 0.011643079109489918
epoch: 24, loss: 0.08565778285264969
epoch: 25, loss: 0.07640542089939117
epoch: 26, loss: 0.038953494280576706
epoch

In [120]:
model.eval()
accuracy_list=[]

with torch.no_grad():
  for x,y in test_loader:
    y_pred=model(x)
    y_pred = (y_pred > 0.8).float()
    batch_accuracy = (y_pred.view(-1) == y).float().mean().item()
    accuracy_list.append(batch_accuracy)

overall_accuracy = sum(accuracy_list) / len(accuracy_list)
print(f'Accuracy: {overall_accuracy:.4f}')


Accuracy: 1.0000
